# Finite element analysis of an elastic frame

______________________________

*Date:* 2018-01-02

*Author:* Pierre Jehel (<pierre.jehel@centralesupelec.fr>)

*Course:* Gestion des Risques Structure (CentraleSupélec, option / MS "Aménagement et Construction Durables")
______________________________

This document presents the principal steps in a finite element analysis program for a simple  structure.
For any FE program, the overall procedure is the same.
The three main steps are:
- Pre-processing: the system is described by the user (inputs)
- Processing: the FE mesh is generated and the problem is numerically solved
- Post-processing: plotting and analysis of the outputs

We first import the Python packages and modules we will need:

In [2]:
import numpy as np
import numpy.matlib
from numpy import linalg
import mesGen
import strMod

## 1. Description of the system

User inputs here the description of the structure she/he wants to model:
- Structure geometry
- Element section geometry
- Element material properties
- Boundary conditions (loading)
- Parameterization for the solution procedure
- Output data

### 1.1. Structure geometry

We consider a 2D frame structure.

We first create an array with, for each structural element, the following list: [node I, node J, element type (1: beam/column, 2: rotational spring), element length, global to local rotation angle] (see figure with the description of the structure):

Nodes coordinates (2D problem):

In [3]:
Lf = 8      # bay length [m]
Hf = 4      # story height [m]
nodCoo = []
nodCoo.append([0,0])     # node 0 [X coordinate,Y coordinate]
nodCoo.append([Lf,0])    # node 1 [X,Y]
nodCoo.append([0,Hf])    # ...
nodCoo.append([Lf,Hf])
nodCoo.append([0,2*Hf])
nodCoo.append([Lf,2*Hf])

Elements connect a node I to a node J; we consider different:
- element types (0->beam/column; 1->rotational spring)
- element sections

In [4]:
strEle = []
strEle.append([0,2,0,0])  # elt 0 [node I, node J, element type, section type] 
strEle.append([2,4,0,0])  # elt 1
strEle.append([1,3,0,1])  # ...
strEle.append([3,5,0,1])
strEle.append([2,3,0,2])
strEle.append([4,5,0,3])

### 1.2. Element properties (material and section geometry)

The geometric properties of selected steel beam sections (see figure) can for instance be found here: <http://ds.arcelormittal.com/repo/Catalog/ArcelorMittal_PV_FR_RU.pdf>

![Geometry of a steel beam section.](img/SteelBeamSection.jpg)

*Remark:* We will consider that the beams can be described using Euler-Bernoulli kinematic assumptions. Other options are generally available in FE analysis programs such as Timoshenko kinematics.

E [N/m2]: material Young's modulus
A [m2]  : element cross-section area
Iy [m4] : element cross-section moment of inertia wrt centroid

In [5]:
eltPro = []
eltPro.append([2.1e11,33.4e-4,2772e-8])   # section type 0  [E, A, Iy] -> IPE 220
eltPro.append([2.1e11,33.4e-4,2772e-8])   # section type 1
eltPro.append([2.1e11,33.4e-4,2772e-8])   # ...
eltPro.append([2.1e11,33.4e-4,2772e-8]) 

### 1.4. Boundary conditions

In 'fixDof' array, we indicate for every node whether the DOFs are free or not.
The line number of the matrix corresponds to the node number.
For each of the 3 DOFs $(u,v,\theta)$, there is a 1 if the DOF is free and a 0 otherwise:

In [6]:
fixDof = []
fixDof.append([0,0,0])   # node 0
fixDof.append([0,0,0])   # node 1
fixDof.append([1,1,1])   # ...
fixDof.append([1,1,1])
fixDof.append([1,1,1])
fixDof.append([1,1,1])

The frame is fixed in the ground at its base. Uniformly distributed static loadings are applied on both beams; self-weight is neglected:

In [7]:
q1 = 1.5e5   # [N] 2nd-floor beam
q2 = 1.5e5   # [N] 3rd-floor beam

In 'staLoa' array, we input external loading actions on nodes:

In [8]:
Fy1 = q1*Lf/2
Mz1 = q1*Lf**2/12
Fy2 = q2*Lf/2
Mz2 = q2*Lf**2/12
bouCon = []
bouCon.append([0,0,0])         # node 0 [Dx, Dy, Rz] (displacements)
bouCon.append([0,0,0])         # node 1 [Dx, Dy, Rz] (displacements)
bouCon.append([0,-Fy1,-Mz1])   # node 2 [Fx, Fy, Mz] (forces)
bouCon.append([0,-Fy1,Mz1])    # ...
bouCon.append([0,-Fy2,-Mz2])
bouCon.append([0,-Fy2,Mz2])

## 2. Mesh generation

In this simple example, the only thing that has to be done for generating the mesh is computing element length and orientation ($cos(\theta)$ and $sin(\theta)$ where $\theta$ is the angle between local and global coordinate systems). We get these mesh data as:

In [9]:
# mesh data
mesDat = mesGen.eltL0(strEle,nodCoo)

# number of free DOFs
nDof = np.sum(fixDof)

## 3. Resolution procedure

We recall that the element stiffness matrix with Euler-Bernoulli kinematics reads:

![Geometry of a steel beam section.](img/EB-beam-K-matrix.pdf)

Let's solve the problem now:

In [10]:
# external forces vector
fe_v = strMod.extLoa(nDof,fixDof,bouCon)
    
# structural stiffness matrix K
K_m = strMod.stiMat(nDof,strEle,mesDat,eltPro,fixDof)

# compute displacements at free DOFs
dis_v = linalg.solve(K_m,fe_v)

We now have the displacements in the global coordinates system (the primary unknowns of the problem) at the free DOFs.

Because there are 3 DOFs at each node (longitudinal and transversal translations and rotation), the $\texttt{dis_v}$ vector is of size $6*3-6=12$ (6 nodes * 3 DOFs/node - 6 frozen DOFs). The frozen DOFs correspond to the nodes fixed in the ground. 

The first 3 elements of vector $\texttt{dis_v}$ are, respectively, the longitudinal and transversal translations and the rotation at node 2; then the elements 4 to 6 of the vector $\texttt{dis_v}$ correspond to the displacements at node 3; and so on.

## 4. Outputs analysis

### 4.1. Internal forces

We are now interested in computing the internal forces in the elements connected to node 2 (elements 0, 1, and 4). This can be computed from the primary unknowns of the problem (the displacements of the free DOFs in the global coordinates system) as follows:
- Retrieve in vector $\texttt{dis_v}$ the displacements at both ends of each element:

In [11]:
# element 0 (column)
dis0_v = np.matlib.zeros((6,1))
dis0_v[0] = 0
dis0_v[1] = 0
dis0_v[2] = 0
dis0_v[3] = dis_v[0]
dis0_v[4] = dis_v[1]
dis0_v[5] = dis_v[2]

# element 1 (column)
dis1_v = np.matlib.zeros((6,1))
dis1_v[0] = dis_v[0]
dis1_v[1] = dis_v[1]
dis1_v[2] = dis_v[2]
dis1_v[3] = dis_v[6]
dis1_v[4] = dis_v[7]
dis1_v[5] = dis_v[8]

# element 4 (beam)
dis4_v = np.matlib.zeros((6,1))
dis4_v[0] = dis_v[0]
dis4_v[1] = dis_v[1]
dis4_v[2] = dis_v[2]
dis4_v[3] = dis_v[3]
dis4_v[4] = dis_v[4]
dis4_v[5] = dis_v[5]

- Project the displacements into the local coordinates system:

In [12]:
# build transformation matrix
# for the columns:
#    Global    -->   Local
#   Y                      x
#   |                      |
#   |          -->         |
#   O----> X        y <----O
cos = 0
sin = 1
R_m = strMod.glo2loc_2D(cos,sin)
        
# displacements in local coordinates system
dis0loc_v = R_m*dis0_v   # column
dis1loc_v = R_m*dis1_v   # column
dis4loc_v = dis4_v       # beam

- Compute the internal forces (normal force, shear force, bendiing moment):

In [13]:
# retrieve stiffness matrices in the local coordinates system
KeCol_m = strMod.beaSti_EB_2D_loc(2.1e11,33.4e-4,Hf,2772e-8)
KeBea_m = strMod.beaSti_EB_2D_loc(2.1e11,33.4e-4,Lf,2772e-8)

# compute internal nodal forces (F=K*d)
forInt0_v = KeCol_m*dis0loc_v   # element 0
forInt1_v = KeCol_m*dis1loc_v   # element 1
forInt4_v = KeBea_m*dis4loc_v   # element 4

In [14]:
# show internal nodal forces
print('F_elmt0 =')
print(forInt0_v)
print('F_elmt1 =')
print(forInt1_v)
print('F_elmt4 =')
print(forInt4_v)

F_elmt0 =
[[ 1200000.        ]
 [  -89278.25303849]
 [ -119877.16952566]
 [-1200000.        ]
 [   89278.25303849]
 [ -237235.84262829]]
F_elmt1 =
[[ 600000.        ]
 [-291581.71751491]
 [-504084.8208204 ]
 [-600000.        ]
 [ 291581.71751491]
 [-662242.04923926]]
F_elmt4 =
[[ -2.02303464e+05]
 [  7.27595761e-12]
 [ -5.86793366e+04]
 [  2.02303464e+05]
 [ -7.27595761e-12]
 [  5.86793366e+04]]


Some comments on these values:
- In each element internal forces vector, the 3 first components are respectively the normal force $N$, the shear force $V$ and the bending moment $M$ at the left end $I$; the 3 last components are the same quantities but at the right end $J$ of the element
- In element 0 (column): $N^0_I=120.0~kN$, $V^0_I = -89.3~kN$, $M^0_I = -120.0~kN.m$, $N^0_J=-120.0~kN$, $V^0_J = 89.3~kN$, $M^0_J = -237.2~kN.m$
- Because element 0 is fixed at its left end $I$, $(-N,-V,-M)^0_I$ correspond to the reactions at node 0. This information is useful for designing the foundations of the structure and the connection at node 0.
- At node 2, the right end of element 0 is connected to the left end of both elements 1 (column) and 4 (beam). We can check that the equilibrium of node 2 is satisfied, that is that the internal forces equilibrite the external action due to the uniformly distributed loading along element 4 (beam):
    - Force along the Y-axis generated by the loading: $F^{ext}_{2,Y} = -q1 \times Lf/2 = -60~kN$
    - Moment around the Z-axis generated by the loading: $M^{ext}_{2,Y} = -q1 \times Lf^2/12 = -800~kN.m$
    - Equilibrium of the forces along the X-axis: $-V^0_J-V^1_I+N^4_I = -89.3+291.6-202.3 = 0$ -> OK
    - Equilibrium of the forces along the Y-axis: $N^0_J+N^1_I+V^4_I = -120.0+60.0+0.0 = -60.0~kN = F^{ext}_{2,Y}$ -> OK
    - Equilibrium of the moments around the Z-axis: $M^0_J+M^1_I+M^4_I = -237.2-504.1-58.7 = -800~kN.m = M^{ext}_{2,Y}$ -> OK

### 4.2. Normal stresses

We can now focus on the normal stresses $\sigma_{xx}(x,y)$ in the structural elements. In an element $e$ of cross-section area $A$, length $L$, and moment of inertia $I_z$ they can be computed from the formula:

\begin{equation}
 \sigma^e_{xx}(x,y) = \frac{N^e(x)}{A} + \frac{M^e(x) \, y}{I_z}
\end{equation}

where

\begin{eqnarray}
 & N^e(x) = \nu(x) + N^e_I\left(1-\frac{x}{L}\right) - N^e_J \frac{x}{L} \\
 & M^e(x) = \mu(x) - M^e_I\left(1-\frac{x}{L}\right) + M^e_J \frac{x}{L}
\end{eqnarray}

with $\nu(x)$ and $\mu(x)$ the normal force and bending moment in the isostatic version of the element $e$. For instance, element 4 is uniformly loaded with distributed vertical load $-q_1$, in this case:
- $\nu(x) = 0$
- $\mu(x) = q_1 x (Lf - x) / 2$